# Binning

In [ ]:
!iqtree -s al_d_root.fasta -T AUTO -ntmax 100 -B 1000


In [ ]:
d = {"14":["14.fq",
                    "/home/gladkov2/storage/Bact_comm_4x/20200225_1435_MN24173_FAK85519_967910e6_3_6_1_barcoded/barcode01/all.fq",
                    "/home/gladkov2/storage/metagenome/bact_4x/new_con/14_F_wa.fq",
                    "/home/gladkov2/storage/metagenome/bact_4x/new_con/14_R_wa.fq"],
     "26":["26.fq",
                    "/home/gladkov2/storage/Bact_comm_4x/20200225_1435_MN24173_FAK85519_967910e6_3_6_1_barcoded/barcode02/all.fq",
                    "/home/gladkov2/storage/metagenome/bact_4x/new_con/26_F_wa.fq",
                    "/home/gladkov2/storage/metagenome/bact_4x/new_con/26_R_wa.fq"],
     "29":["29.fq",
                    "/home/gladkov2/storage/Bact_comm_4x/20200225_1435_MN24173_FAK85519_967910e6_3_6_1_barcoded/barcode03/all.fq",
                    "/home/gladkov2/storage/metagenome/bact_4x/new_con/29_F_wa.fq",
                    "/home/gladkov2/storage/metagenome/bact_4x/new_con/29_R_wa.fq"],
     "46":["46.fq",
                    "/home/gladkov2/storage/Bact_comm_4x/20200225_1435_MN24173_FAK85519_967910e6_3_6_1_barcoded/barcode04/all.fq",
                    "/home/gladkov2/storage/metagenome/bact_4x/new_con/46_F_wa.fq",
                    "/home/gladkov2/storage/metagenome/bact_4x/new_con/46_R_wa.fq"],
    }

for f in d:
    con = d[f][0]
    mi_f = d[f][2]
    mi_r = d[f][3]
    ont = d[f][1]
    !sed -n '1~4s/^@/>/p;2~4p' $ont > $f\_ont.fasta
    !minimap2 -ax map-ont $con  $f\_ont.fasta $mi_f $mi_r -t 50  > $f\_bwa.sam
    !samtools sort $f\_bwa.sam > $f\_bwa_sorted.bam
  
for f in d:
    print(f'\n{f}')
    !samtools flagstat $f\_bwa_sorted.bam

for f in d:
    !jgi_summarize_bam_contig_depths \
    --outputDepth $f\_depth.txt \
    --referenceFasta $f\.fq \
    $f\_bwa_sorted.bam

for f in d:
    !metabat2 -i $f\.fq -o mags/$f\_gen_cov -a  $f\_depth.txt
    !metabat2 -i $f\.fq -o mags/$f\_gen_w_cov
!checkm lineage_wf -x fa  -f checkm/N2_MAGs_checkm.tab -t 50 mags/ checkm/
!checkm qa ./checkm/lineage.ms ./checkm -o 2 -f res --tab_table -t 50
    


In [ ]:
df = pd.read_csv("res", delimiter="\t")
df.columns
df = df[['Bin Id','Marker lineage', 'Completeness','Contamination', 'Strain heterogeneity',
       'Genome size (bp)', 'N50 (contigs)', 'GC']]
df_h = df.query("Completeness > 90 and Contamination < 5")
df_h.to_csv('checkm_stat_hq.tsv', index=False, sep='\t')

In [ ]:
df_h = df.query("Completeness > 90 and Contamination < 5")

with open('high_q_bins_list.txt', 'a') as f:
    dfAsString = df_h['Bin Id'].to_string(header=False, index=False)
    f.write(dfAsString)

### Kraken2 processing with GTDB

In [ ]:
%%bash

mkdir kraken2_gdtb_res
for file in ~/storage/metagenome/bact_4x/new_con/diamond/con/*[0-9].faa; do
    mkdir temp
    fname=$(basename "$file" .faa)
    kraken2 --report kraken2_gdtb_res/${fname}_report_gtdb --db /home/gladkov2/storage/somebases/kraken_gtdb_202_struo2/ --threads 50 $file > temp/${fname}_res_kraken.tsv
    cat temp/${fname}_res_kraken.tsv | cut -f3 | cut -f3 |taxonkit --data-dir ~/storage/somebases/kraken_gtdb_202_struo2/taxonomy/  -j 50 lineage | taxonkit --data-dir ~/storage/somebases/kraken_gtdb_202_struo2/taxonomy/  -j 50 reformat  -I 1 -r "Unassigned" -f "{k}\t{p}\t{c}\t{o}\t{f}\t{g}\t{s}" > temp/first 
    cat temp/${fname}_res_kraken.tsv | cut -f2 > temp/second
    paste temp/second temp/first > kraken2_gdtb_res/${fname}_res_kraken.tsv
    rm -r temp
done;

In [ ]:
!python /home/gladkov2/storage/apps/KrakenTools/combine_kreports.py \
    -r kraken2_gdtb_res/14_report_gtdb kraken2_gdtb_res/26_report_gtdb kraken2_gdtb_res/29_report_gtdb kraken2_gdtb_res/46_report_gtdb \
    --display-headers \
    --sample-names c14 c26 c29 c46 \
    -o kraken2_gdtb_res/all.kreport2 

!python /home/gladkov2/storage/apps/KrakenTools/kreport2krona.py -r kraken2_gdtb_res/all.kreport2 -o kraken2_gdtb_res/all.krona
!ktImportText kraken2_gdtb_res/all.krona -o kraken2_gdtb_res/all.krona.html

In [ ]:
%%bash

for file in kraken2_gdtb_res/*report_gtdb; do
    fname=$(basename "$file")
    fname=${fname%%_*}
    echo $fname
    echo $file
    python /home/gladkov2/storage/apps/KrakenTools/kreport2krona.py -r ${file} -o kraken2_gdtb_res/${fname}.krona
    sed -i 's/.__.__//g' kraken2_gdtb_res/${fname}.krona
done;

ktImportText kraken2_gdtb_res/*.krona -o kraken2_gdtb_res/all.krona.html

## Clusters

CGC-Finder


In [ ]:
%%bash

run_dbcan 26.faa meta --cluster [135/1916]
dia_cpu 20 --tf_cpu 20 --hmm_cpu 20 --stp_cpu 20 --tools diamond --out_dir db_26 --db_dir ~/storage/someb
ases/dbcan 
